# Recurrent Networks Basic Architecture

### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import exp
from numpy import newaxis
import copy
%matplotlib inline 

### Transfer Functions

In [2]:
def hardlim(n):
    # w and p are vectors of length n and b is the bias
    # hardlim return 1 if sum w*p + b is greater than or equal to zero and returns 0 otherwise
    if n >= 0:
        return 1
    else:
        return 0

def hardlims(n):
    # w and p are vectors of length n and b is the bias
    # hardlim returns 1 if sum w*p + b is greater than or equal to zero and returns -1 otherwise
    if n >= 0:
        return 1
    else:
        return -1    

def purelin(n):
    # w and p are vectors of length n and b is the bias
    # purelin simply return the sum w*p + bias
    return n

def satlin(n):
    # w and p are vectors of length n and b is the bias
    # satlin returns 0 if sum w*p + b is less than zero, returns sum itself if sum is less than or equal to one
    # and 1 is sum is greater than 1
    a = n
    if a < 0:
        return 0
    elif a <= 1:
        return a
    else: 
        return 1
    
def satlins(n):
    # w and p are vectors of length n and b is the bias
    # satlin returns -1 if sum w*p + b is less than -1, returns sum itself if sum is less than or equal to one
    # and 1 is sum is greater than 1
    a = n
    if a < -1:
        return -1
    elif a <= 1:
        return a
    else: 
        return 1
    
def logsig(n):
    # w and p are vectors of length n and b is the bias
    # logsig returns 1/(1+e^-sum) where sum is w*p + b
    return 1.0/(1.0 + exp(-1.0*n))

def tansig(n):
    # w and p are vectors of length n and b is the bias
    # tansig returns (e^sum - e^-sum)/(e^sum + s^-sum) where sum is w*p + b
    return (exp(n) - exp(-n))/(exp(n) + exp(-n))

def test(func):
    n = -5
    x = []
    y = []
    precision = 1000
    for i in range(0, 10*precision):
        x.append(n + 1.0*i/precision)
        y.append(func(n + 1.0*i/precision))
    plt.xlim([-5, 5])
    plt.ylim([-1.5, 1.5])
    plt.xlabel('Summer Output')
    plt.ylabel(func)
    plt.plot(x, y)
    plt.show()
    print(np.array(x))
    print(np.array(y))

#test(hardlim)
#test(hardlims)
#test(purelin)
#test(satlin)
#test(satlins)
#test(logsig)
#test(tansig)

### Single Layer and Multiple Layer Network

In [3]:
def getNeuronOuput(f, w, b, p):
    # w and p are vectors of length n and b is the bias
    # summer returns w_1*p_1 + w_2*p_2 + ... + w_n*p_n + b
    # VARIABLES MUST FOLLOW THE FOLLOWING CONDITIONS...
    # f IS THE TRANSFER FUNCTION OF NEURON
    # w IS A 1-D NUMPY ARRAY CONTAINING WEIGHTS OF NEURON
    # b IS THE BIAS OF NEURON
    # p IS A 1-D NUMPY ARRAY CONTAINING INPUT TO NERUON
    # THE VALUE OF NEURON OUTPUT IS RETURNED
    n = sum(w*p) + b
    return f(n)

# TEST
# f = tansig
# w = np.array([0.5, 1, 1])
# b = 1
# p = np.array([-1, -1, 1])
# print(getNeuronOuput(f, w, b, p))

def apply_on_2D(f, n):           # It is called 2D because it is used for 2D weight matrices.
    # f is numpy array of column vector of functions 
    # n is numpy array of column vector of neurons' outputs in layer.
    output = n.copy()
    S = len(f)
    for i in range(S):
        output[i][0] = f[i][0](n[i][0])
    return output

# TEST 
# f = np.array([logsig, tansig])[:, newaxis]
# p = np.array([2.0, 0.3])[:, newaxis]
# print(apply_on_2D(f, p))

def getLayerOutput(f, W, b, p):
    # S is the number of neurons in the layer, R is the number of inputs from previous layer. 
    # funcs or f is the function matrix (Sx1), W is the weight matrix (SxR), inputs or p is the input matrix,
    # biasses or b is the bias matrix.
    # getLayerOuput returns the result (matrix a) obtained by operating functions from funtions matrix 
    # on weights times inputs plus bias. 
    # a = f(n), n = W*p + b, return a. OR n[i] = f[i](W[i]*p + b[i])
    # VARIABLES MUST FOLLOW THE FOLLOWING CONDITIONS...
    # f IS A NUMPY ARRAY OF COLUMN VECTOR OF TRANSFER FUNCTIONS OF NEURONS
    # W IS A NUMPY ARRAY OF WEIGHTS OF NEURONS IN LAYER
    # b IS A NUMPY ARRAY OF COLUMN VECTORS OF BIASES OF NEURONS 
    # p IS A NUMPY ARRAY OF COLUMN VECTOR OF INPUTS TO THE LAYER
    n = W.dot(p) + b
    layerOutput = apply_on_2D(f, n)
    return layerOutput

# TEST
# f = np.array([logsig, hardlim])[:, newaxis]
# W = np.array([[1, 3, 4], 
#               [6, 2, 1]])
# b = np.array([1, 2])[:, newaxis]
# p = np.array([1, 2, 3])[:, newaxis]
# print(getLayerOutput(f, W, b, p))

def getMultipleLayerOutput(f, W, b, p):
    # Let k be the number of layers in neural network. f is a two dimensional matrix f with 
    # f_1, f_2, ... , f_(k) as its elements. W is the three dimensional vector with W_1, W_2, ... , W_(k) 
    # as its elements. b is a two dimensinal matrix with b_1, b_2, ... , b_(k) as its 
    # elements. p is the input vector. getMultipleLayerOutput takes
    # the input p as the input to network and layers of neurons operator consecutively on
    # it to get the output of the network. It returns THE FINAL OUTPUT OF ALL THE LAYERS.
    # a be the two dimensional matrix with a_1, a_2, ..., a_(k) as outputs of 
    # the layers of neural networks. a[0] = f[0](W[0]*p + b[0]) and a[i] = f[i](W[i]*a[i-1] + b[i]) for i > 0.
    # VARIABLES MUST FOLLOW THE FOLLOWING CONDITIONS...
    # f IS A LIST OF NUMPY ARRAYS OF COLUMN VECTORS OF TRANSFER FUNCTIONS OF LAYERS
    # W IS A LIST OF NUMPY ARRAY OF WEIGHTS OF NEURONS IN LAYER
    # b IS A LIST OF NUMPY ARRAY OF COLUMN VECTORS OF BIASES OF NEURONS 
    # p IS A NUMPY ARRAY OF COLUMN VECTOR OF INPUTS TO THE LAYER
    a = getLayerOutput(np.array(f[0]), np.array(W[0]), np.array(b[0]), np.array(p))
    for i in range(1, len(W)):
        a = getLayerOutput(np.array(f[i]), np.array(W[i]), np.array(b[i]), np.array(a))
    return a

# TEST
# f = [np.array([logsig]*4)[:, newaxis],
#      np.array([tansig]*2)[:, newaxis],
#      np.array([hardlim]*3)[:, newaxis]]
# W = [np.array([[1, 1, 1], 
#               [1, 5, 2], 
#               [-1, -2, 4], 
#               [2, 1, 2]]),
#      np.array([[1, 10, 1, 1], 
#                [-1, -2, -3, -2]]),
#      np.array([[2, -2], 
#               [-1, 3], 
#               [6, -3]])]
# b = [np.array([1, -2, 3, -4])[:, newaxis],
#      np.array([1, -1])[:, newaxis],
#      np.array([1, -2, 0])[:, newaxis]]
# p = np.array([1, 2, 3])[:, newaxis]
# print(getMultipleLayerOutput(f, W, b, p))

### Class for Multilayered Neural Network

In [4]:
class MultilayeredNetwork(object):
    # Let there be k layers (numbered 0, 1, ... , k-1). 
    # Class contains the following variables:
    # functions - functions[i][j] is the transfer function of jth neuron in ith layer.
    # weights - weights[i][j][l] is kth weight 
    #    for jth neuron in lth layer.
    # biases - biases[i][j] is the bias of jth neuron in ith layer.
    
    def __init__(self, functions, weights, biases, learningRate):
        # WHILE CREATING AN INSTANCE OF MultilayeredNetwork, CREATE VARIABLES AS FOLLOWS...
        # functions SHOULD BE A LIST OF NUMPY ARRAY OF COLUMN VECTORS FOR EACH LAYER
        # weights SHOULD BE A LIST OF NUMPY ARRAYS OF WEIGHTS FOR EACH LAYER
        # biases SHOULD BE A LIST OF NUMPY ARRAYS OF COLUMN VECTORS FOR EACH LAYER
        # learningRate SHOULD BE A FLOAT OR DOUBLE
        self.functions = functions
        self.weights = weights
        self.biases = biases
        self.learningRate = learningRate
        return

    def getNumberOfLayers(self):
        return len(self.weights)
    
    def getNumOfNeurons(self, i):
        # returns the number of neurons in ith layer
        return len(self.weights[i])
    
    def getInputSize(self, i):
        # returns the size of input in network to ith layer
        return len(self.weights[i][0])
    
    def getFunctions(self, i):
        # returns the transfer functions of the ith layer
        return self.functions[i]
    
    def getWeights(self):
        return self.weights
   
    def getBiases(self):
        return self.biases
    
    def getLearningRate(self):
        return self.learningRate
       
    def getOutput(self, inputToNet):
        # inputToNet SHOULD BE A COLUMN VECTOR EQUAL TO THE SIXE OF INPUT OF FIRST LAYER
        return getMultipleLayerOutput(self.functions, self.weights, self.biases, inputToNet)
    
# TEST
# functions = [np.array([logsig]*4)[:, newaxis], 
#              np.array([tansig]*2)[:, newaxis], 
#              np.array([hardlim]*3)[:, newaxis]]
# weights = [np.array([[1, 1, 1], 
#                      [1, 5, 2], 
#                      [-1, -2, 4], 
#                      [2, 1, 2]]),
#            np.array([[1, 10, 1, 1], 
#                      [-1, -2, -3, -2]]),
#            np.array([[2, -2], 
#                      [-1, 3], 
#                      [6, -3]])]
# biases = [np.array([1, -2, 3, -4])[:, newaxis],
#           np.array([1, -1])[:, newaxis],
#           np.array([1, -2, 0])[:, newaxis]]
# p = np.array([1, 2, 3])[:, newaxis]

# multiLayerTest = MultilayeredNetwork(functions, weights, biases, 0.02)
# print(multiLayerTest.getWeights())
# print(multiLayerTest.getOutput(p))

### Delay Function

If u(t) is the input to delay function, it's output is a(t) = u(t-1). a(0) is provided as the initialization input.

### Integrator Function

Input to the integrator is u(t). Ouput is a(t) = integration(u(t)) + a(0) where a(0) is the initialization input to integrator function.

### Reccurent Network

#### Discrete-Time Recurrent Network

a(t+1) = satlins(W*a(t) + b)

Since these networks have memory, for different networks, we have to create a class which holds the data together.

In [5]:
class RecurrentNetwork(object):
    # VARIABLES: inputSize, size, storedInput, weights, biases, time
    # inputSize - the size of input to network = size of input to each neuron
    # size - size = number of neurons in the recurrent network
    # storedInput - a vector of size S containing which neurons remember
    # weights - weights[i][j] is the jth weight of ith neuron
    # biases - biases[i] is the bias of the ith neuron
    # time - time (in units of outputs taken) it has passed since the network was created
    
    def __init__(self, weights, biases, initInput):
        # WHILE CREATING AN INSTANCE OF RecurrentNetwork, CREATE VARIABLES AS FOLLOWS...
        # weights SHOULD BE A NUMPY ARRAY OF WEIGHTS FOR LAYER
        # biases SHOULD BE A NUMPY ARRAY OF COLUMN VECTORS FOR LAYER
        # initInput SHOULD BE A NUMPY ARRAY OF COLUMN VECTOR 
        self.storedInput = initInput
        self.weights = weights
        self.biases = biases
        self.time = 0
        return
    
    def getSize(self):
        return len(weights)
    
    def getInputSize(self):
        return len(weights[0])
    
    def getStoredInput(self):
        return self.storedInput
    
    def getWeights(self):
        return self.weights
    
    def getBiases(self):
        return self.biases
    
    def getTime(self):
        return self.time
    
    def getRecurrentOutput(self):
        oldOutput = copy.copy(self.storedInput)
        self.time = self.time + 1
        # storedInput = weights * storedInput + biases
        self.storedInput = getLayerOutput(np.array([satlins]*self.getSize())[:, newaxis], 
                                          self.weights, 
                                          self.biases, 
                                          self.storedInput)
        return oldOutput

# TEST
weights = np.array([[1, -2, 3, 4], 
                    [1, 2, -3, 4], 
                    [-1, 2, 3, 4], 
                    [1, 2, 3, -4]])
biases = np.array([-1, 4, -5, 6])[:, newaxis]
initInput = np.array([0.2, 0.33, -0.2, -0.5])[:, newaxis]
a = RecurrentNetwork(weights, biases, initInput)
for i in range(0, 10):
    print(a.getTime())
    print(a.getStoredInput())
    print(a.getRecurrentOutput())
    print("x---X---X")

0
[[ 0.2 ]
 [ 0.33]
 [-0.2 ]
 [-0.5 ]]
[[ 0.2 ]
 [ 0.33]
 [-0.2 ]
 [-0.5 ]]
x---X---X
1
[[-1.]
 [ 1.]
 [-1.]
 [ 1.]]
[[-1.]
 [ 1.]
 [-1.]
 [ 1.]]
x---X---X
2
[[-1.]
 [ 1.]
 [-1.]
 [ 0.]]
[[-1.]
 [ 1.]
 [-1.]
 [ 0.]]
x---X---X
3
[[-1.]
 [ 1.]
 [-1.]
 [ 1.]]
[[-1.]
 [ 1.]
 [-1.]
 [ 1.]]
x---X---X
4
[[-1.]
 [ 1.]
 [-1.]
 [ 0.]]
[[-1.]
 [ 1.]
 [-1.]
 [ 0.]]
x---X---X
5
[[-1.]
 [ 1.]
 [-1.]
 [ 1.]]
[[-1.]
 [ 1.]
 [-1.]
 [ 1.]]
x---X---X
6
[[-1.]
 [ 1.]
 [-1.]
 [ 0.]]
[[-1.]
 [ 1.]
 [-1.]
 [ 0.]]
x---X---X
7
[[-1.]
 [ 1.]
 [-1.]
 [ 1.]]
[[-1.]
 [ 1.]
 [-1.]
 [ 1.]]
x---X---X
8
[[-1.]
 [ 1.]
 [-1.]
 [ 0.]]
[[-1.]
 [ 1.]
 [-1.]
 [ 0.]]
x---X---X
9
[[-1.]
 [ 1.]
 [-1.]
 [ 1.]]
[[-1.]
 [ 1.]
 [-1.]
 [ 1.]]
x---X---X
